In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!unzip '/content/gdrive/My Drive/Train.zip'
!unzip '/content/gdrive/My Drive/Test.zip'
!unzip '/content/gdrive/My Drive/Val.zip'

Archive:  /content/gdrive/My Drive/Train.zip
   creating: Train/
   creating: Train/Train_images/
  inflating: Train/Train_images/5_Ortho.tif  
  inflating: Train/Train_images/26_Ortho.tif  
  inflating: Train/Train_images/38_Ortho.tif  
  inflating: Train/Train_images/150_Ortho.tif  
  inflating: Train/Train_images/181_Ortho.tif  
  inflating: Train/Train_images/183_Ortho.tif  
  inflating: Train/Train_images/190_Ortho.tif  
  inflating: Train/Train_images/200_Ortho.tif  
  inflating: Train/Train_images/202_Ortho.tif  
  inflating: Train/Train_images/205_Ortho.tif  
  inflating: Train/Train_images/207_Ortho.tif  
  inflating: Train/Train_images/242_Ortho.tif  
  inflating: Train/Train_images/248_Ortho.tif  
  inflating: Train/Train_images/255_Ortho.tif  
  inflating: Train/Train_images/259_Ortho.tif  
  inflating: Train/Train_images/271_Ortho.tif  
  inflating: Train/Train_images/284_Ortho.tif  
  inflating: Train/Train_images/286_Ortho.tif  
  inflating: Train/Train_images/288_Ortho.

In [0]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
import random
import numpy as np
import matplotlib.pyplot as plt
from keras import backend as K
from keras.layers import Input, Conv2D, MaxPooling2D, concatenate, UpSampling2D
from keras.optimizers import Adadelta, Nadam
from keras.models import Model, load_model
from keras.utils import multi_gpu_model, plot_model
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.preprocessing import image
import tensorflow as tf
from keras.losses import binary_crossentropy
#from multiclassunet import Unet
import tqdm
import cv2
from keras.utils import Sequence
from keras.callbacks import Callback
#from dilatednet import DilatedNet



# image_dir = "/home/kongbe/Downloads/Data/Train/Train_images"
# mask_dir = "/home/kongbe/Downloads/Data/Train/Train_segmentation"
image_dir = "/content/Train/Train_images"
mask_dir = "/content/Train/Train_segmentation"
val_dir = "/content/Val/Val_images"
mask_val_dir = "/content/Val/Val_segmentation"
test_dir = "/content/Test/Test_images"
mask_test_dir = "/content/Test/Test_segmentation"

# image_list = os.listdir(image_dir)
# mask_list = os.listdir(mask_dir)
# val_list = os.listdir(val_dir)
# mask_val_list = os.listdir(mask_val_dir)

COLOR_DICT = {'Building' : [0,0,255] , 'Grass' :[0,255,255], 'Tree' :[0,255,0] , 'Car':[255,255,0] , 'Sand':[255,0,0],
                        'Road':[255,255,255], 'Unlabeled':[0,0,0] }

dict_label = {'Building' : 29 , 'Grass' :178, 'Tree' : 149 , 'Car':225 , 'Sand':76,
                        'Road':255, 'Unlabeled':0 }

Grass =[0,0,255]
Building=[0,255,255]
Tree=[0,255,0]
Car=[255,255,0]
Sand=[255,0,0]
Road=[255,255,255]
COLOR_DICT = np.array([Grass, Building, Tree, Car, Sand,Road])
label=[29,179,149,225,76,255]

# Setting general
batch_size = 4
img_height, img_width = 256, 256
classes = 6
filters_n = 64

# In Train folder
samples = 20000
steps = samples//batch_size
###

# In Val folder
val_samples = 1000
val_steps = val_samples//batch_size
###

# In test folder
test_samples = 888
test_steps = test_samples//batch_size
###


# Preprocessing

class seg_gen(Sequence):
    def __init__(self, x_dir, y_dir, batch_size,start=1, end=20000):
        self.x, self.y = x_dir, y_dir
        self.batch_size = batch_size
        self.start, self.end = start, end
    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        idx = np.random.randint(self.start, self.end, batch_size)
        batch_x, batch_y = [], []
        drawn = 0
        for i in idx:
            _image = image.img_to_array(image.load_img(f'{self.x}/{i}_Ortho.tif', color_mode='grayscale', target_size=(img_height, img_width)))/255.   
            img = image.img_to_array(image.load_img(f'{self.y}/{i}_Label.tif', color_mode = 'grayscale', target_size=(img_height, img_width)))
            labels = np.unique(img)
            # if len(labels) < 3:
            #     idx = np.random.randint(0, 300, batch_size-drawn)
            #     continue

            img = np.squeeze(img)
            mask = np.zeros((img.shape[0], img.shape[1], 6))
            for i in labels : # Labels is all gray values [29,178,149,225,76,255]
                if i == dict_label['Building']:      # dict_label is a dictionary of gray value for label
                    mask[:,:,0] = np.logical_or(mask[:,:,0],(img==i))
                elif i == dict_label['Grass']:
                    mask[:,:,1] = np.logical_or(mask[:,:,1],(img==i))
                elif i == dict_label['Tree']:
                    mask[:,:,2] = np.logical_or(mask[:,:,2],(img==i))
                elif i == dict_label['Car']:
                    mask[:,:,3] = np.logical_or(mask[:,:,3],(img==i))
                elif i == dict_label['Sand']:
                    mask[:,:,4] = np.logical_or(mask[:,:,4],(img==i))
                elif i == dict_label['Road']:
                    mask[:,:,5] = np.logical_or(mask[:,:,5],(img==i))
                   
            
            #mask = np.resize(mask,(img_height*img_width, 6))
          
            
            batch_y.append(mask)
            batch_x.append(_image)
            drawn += 1

        return np.array(batch_x), np.array(batch_y)

Using TensorFlow backend.


In [0]:
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [0]:
import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras

 
def unet(pretrained_weights = None,input_size = (256,256,1)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(6, 1, activation = 'softmax')(conv9)

    model = Model(input = inputs, output = conv10)
    epochs = 50
    learning_rate = 0.0005
    decay_rate = learning_rate / epochs
    opt = Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=decay_rate, amsgrad=False)
    model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy',f1_m,precision_m, recall_m])
    #model.compile(optimizer = Adam(lr = 1e-4), loss = 'categorical_crossentropy', metrics = ['accuracy'])
    
    #model.summary()

    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model


In [0]:
#from model import *
# from Model import *
# from data import *
# from batch_training import *

#train_gen = seg_gen('/home/kongbe/Downloads/Data/Train/Train_images','/home/kongbe/Downloads/Data/Train/Train_segmentation',64)
train_gen = seg_gen(image_dir, mask_dir, batch_size, start = 1, end = 20000 )
val_gen = seg_gen(val_dir, mask_val_dir, batch_size, start = 20001, end = 21000)
test_gen = seg_gen(test_dir, mask_test_dir, batch_size, start = 21001, end = 21888)

#myGene = trainGenerator(64,'/home/kongbe/Downloads/Data/Train','Train_image','Train_segmentation',save_to_dir = None)
model = unet()
model.summary()







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 256, 256, 64) 36928       conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 128, 128, 64) 0           co

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:103: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`


In [0]:
val_gen.__len__    ##  Check what the validation set is generatored by Seg_gen ?

<bound method seg_gen.__len__ of <__main__.seg_gen object at 0x7f81a0596cf8>>

In [0]:
#model.load_weights("/content/gdrive/My Drive/my_model_6.hdf5")

In [0]:
model_checkpoint = ModelCheckpoint('/content/gdrive/My Drive/my_model_6.hdf5', monitor='val_loss',verbose=1, save_best_only=False, save_weights_only=True)

model.fit_generator(train_gen, validation_data = val_gen, validation_steps = val_steps, steps_per_epoch=steps, epochs=5,
                   callbacks=[model_checkpoint], workers=16, use_multiprocessing=True)
#model.fit_generator(train_gen, steps_per_epoch=250, epochs=3, callbacks=[model_checkpoint], workers=16, use_multiprocessing=True)
# testGene = testGenerator("/home/kongbe/Downloads/Data/Test
# results = model.predict_generator(testGene,30,verbose=1)
# saveResult("/home/kongbe/Downloads/Data/Result",results)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/5
 713/5000 [===>..........................] - ETA: 24:45 - loss: 0.0787 - acc: 0.9686

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

Process ForkPoolWorker-1676:
Process ForkPoolWorker-1650:
Process ForkPoolWorker-1672:
Process ForkPoolWorker-1670:
Process ForkPoolWorker-1678:
Process ForkPoolWorker-1660:
Process ForkPoolWorker-1675:
Process ForkPoolWorker-1656:
Process ForkPoolWorker-1668:
Process ForkPoolWorker-1654:
Process ForkPoolWorker-1652:
Process ForkPoolWorker-1662:
Process ForkPoolWorker-1680:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-1666:
Traceback (most recent call last):
Process ForkPoolWorker-1658:
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Process ForkPoolWorker-1664:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most rece

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-38ef990ee804>", line 4, in <module>
    callbacks=[model_checkpoint], workers=16, use_multiprocessing=True)
  File "/usr/local/lib/python3.6/dist-packages/keras/legacy/interfaces.py", line 91, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/keras/engine/training.py", line 1658, in fit_generator
    initial_epoch=initial_epoch)
  File "/usr/local/lib/python3.6/dist-packages/keras/engine/training_generator.py", line 215, in fit_generator
    class_weight=class_weight)
  File "/usr/local/lib/python3.6/dist-packages/keras/engine/training.py", line 1449, in train_on_batch
    outputs = self.train_function(ins)
  File "/usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py", line 2979, in __call__
    return self._

KeyboardInterrupt: ignored

In [0]:
# Evaluate in test data
loss, score, f1_score, precision, recall = model.evaluate_generator(val_gen, steps = val_steps, workers = 16, use_multiprocessing = True )

In [0]:
loss, score

(0.5640053749084473, 0.84002685546875)

In [0]:
f1_score

0.8432165384292603

In [0]:
precision

0.852962851524353

In [0]:
recall

0.8336906433105469

In [0]:
# List all classes of predict image 
def classes(mask_predict): # mask_predict has shape (256,256,6) , because it's go through the transfer_mask function to transfer one-hot form
  mask_predict = mask_predict.flatten()
  classes = []
  for i in range(0,len(mask_predict),6):
    if mask_predict[i]==1 : 
      classes.append('Building')
    if mask_predict[i+1]==1 :
      classes.append('Grass')
    if mask_predict[i+2]==1 :
      classes.append('Tree')
    if mask_predict[i+3]==1 :
      classes.append('Car')
    if mask_predict[i+4]==1 :
      classes.append('Sand')   
    if mask_predict[i+5]==1 :
      classes.append('Road')
  return classes    # classes là list gồm các lớp được liệt kê qua 1 lần chạy mask_predict (1 ảnh)

  print('Image have :', {i for i in classes}) #in ra ảnh có mấy lớp



In [0]:
# Tính toán diện tích lớp được quan tâm
# Lý thuyết : Pixel là phần tử nhỏ nhất cấu thành nên ảnh và video trong các thiết bị điện tử 
# nên px có mối quan hệ với các đại lượng đo chiều dài khác . Nhưng px không mang một mối quan hệ xác đinh
# nào cả . Nên ta cần chuẩn hóa như sau. Với bài toán đang làm thì thông số người ta đưa ra 1 ảnh Ortho(RGB) 
# có kích thước 6k*6k*3 , trong đó 1px = 0.012 mm . => cần chuẩn hóa về mối quan hệ này.
# Tính toán S trên ảnh phân vùng :
# S_Building = Tổng số pixel mà phân vùng Building đó chiếm = n * 1px * 1px = n * 0.012^2 (mm) trong đó n là số px của phần vùng BUilding
# Ngoài ra người ta còn cung cấp với mức chuẩn hóa đã cho (1px = 0.012mm) thì Ty le anh chup ve tinh 1 : 8 000 000 
# ==> S_Building_real = S_Building * 8 000 000 = ??? (mm^2)
def Square(array_mask):  # shape array_mask = (256,256,6)
  S, px2 = [], 0.012*0.012
  #array_mask = array_mask.reshape(array_mask, (256,256,6))
  S_Building = np.sum(array_mask[...,0])*8000000*px2   # (mm^2)
  S_Road = np.sum(array_mask[...,5])*8000000*px2       # (mm^2)
  S_Tree = np.sum(array_mask[...,2])*8000000*px2       # (mm^2)

  return [S_Building, S_Road, S_Tree]



In [0]:
def testGenerator(test_path,num_image = 100,target_size = (256,256),as_gray = True):
    for i in range(num_image):
        img = io.imread(os.path.join(test_path,"%d_Ortho.tif"%(i+21001)))
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        gray = gray / 255
        gray = np.reshape(gray,(1,)+gray.shape+(1,))
        yield gray

def labelVisualize(num_class,color_dict,img):
    img = np.resize(img, (256,256,6)) 
    for i in range(256):
      for j in range(256):
        tmp = img[i][j]
        new_label = [int(k==np.max(tmp)) for k in tmp]
        img[i][j] = np.array(new_label)
    mask_RGB = np.zeros((img.shape[0],img.shape[1]) + (3,))
    mask_gray = np.sum(img*label, axis=2)
    for i in range(num_class):
      idx=np.where(mask_gray==label[i])
      mask_RGB[idx]=COLOR_DICT[i]
    return mask_RGB/255

def saveResult(save_path,npyfile,flag_multi_class = True,num_class = 6):
    for i,item in enumerate(npyfile):
        img = labelVisualize(num_class,COLOR_DICT,item)#img segmentation
        #cái này là kết quả cuối cùng
        io.imsave(os.path.join(save_path,"%d_predict.png"%(i+21001)),img)#save result

def saveText(save_path, npyfile):
  for n,img in enumerate(npyfile):
    img = np.resize(img, (256,256,6)) 
    for i in range(256):
      for j in range(256):
        tmp = img[i][j]
        new_label = [int(k==np.max(tmp)) for k in tmp]
        img[i][j] = np.array(new_label)
    S = Square(img)
    dict ={i for i in classes(img)}

    f = open(f"/content/gdrive/My Drive/Predict/{n+21001}_descibre.txt", "w+")
    f.write(f"{n+21001}th image \nImage have : {dict} \nAbout image:S_Building={S[0]}, S_Road={S[1]}, S_Tree={S[2]}")
    f.close()

In [0]:
###
# Predict and compare
model.load_weights("/content/gdrive/My Drive/my_model_6.hdf5")
#idx = np.random.randint(21001,21888,1)
idx = 21006


In [0]:
testGene = testGenerator(r"/content/Test/Test_images")#prepare data
results = model.predict_generator(testGene,100,verbose=1)#predict
saveResult(r"/content/gdrive/My Drive/Predict",results)#save results to path
saveText(r"/content/gdrive/My Drive/Predict",results) #save text to path

100/100 [==============================] - 3s 31ms/step


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: UserWarning: /content/gdrive/My Drive/Predict/21004_predict.png is a low contrast image
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: UserWarning: /content/gdrive/My Drive/Predict/21010_predict.png is a low contrast image


In [0]:
def saveResult_true(save_path,test_path,num_image = 100,flag_multi_class = True,num_class = 6):
    for i in range(num_image):
        img = io.imread(os.path.join(test_path,"%d_Label.tif"%(i+21001)))
        io.imsave(os.path.join(save_path,"%d_true.png"%(i+21001)),img)#save ảnh label đúng

saveResult_true(r"/content/gdrive/My Drive/Predict",'/content/Test/Test_segmentation')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: /content/gdrive/My Drive/Predict/21004_true.png is a low contrast image
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: /content/gdrive/My Drive/Predict/21010_true.png is a low contrast image
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: /content/gdrive/My Drive/Predict/21017_true.png is a low contrast image
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: /content/gdrive/My Drive/Predict/21018_true.png is a low contrast image
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: /content/gdrive/My Drive/Predict/21019_true.png is a low contrast image
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launch

In [0]:
def save_Ortho_image(save_path,test_path,num_image = 10,flag_multi_class = True,num_class = 6):
    for i in range(num_image):
        img = io.imread(os.path.join(test_path,"%d_Ortho.tif"%(i+21001)))
        io.imsave(os.path.join(save_path,"%d_Ortho.png"%(i+21001)),img)#save ảnh test

save_Ortho_image(r"/content/gdrive/My Drive/Predict",'/content/Test/Test_images')